# 深層学習

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークでマルコフモデルを作成し、深層学習における言語モデル実装の基礎を学ぶ。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflowのログを非表示
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [2]:
with tf.device('/cpu:0'): # 何故か勝手にGPUが使われるのでこちらからCPUを指定
    ds = tfds.load('wiki40b/ja', split='test', )
ds = list(ds.as_numpy_iterator())

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:100]


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、本節以降トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。各文章をトークンのリストとして格納する。

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [6]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [7]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [8]:
v = build_vocab_from_iterator([
    ['私', 'は', 'りんご', 'が', '好き'],
    ['私', 'は', 'バナナ', 'を', '食べる'],
    ['今日', 'は', '晴れ', 'です']
])
v.get_itos() # vocabrary

['は', '私', 'が', 'です', 'りんご', 'を', 'バナナ', '今日', '好き', '晴れ', '食べる']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

print('num of vocabrary:', len(v))
data_ids[0][:10] # example

num of vocabrary: 4602


[17, 3681, 3742, 2, 4, 3873, 4415, 47, 19, 70]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for id1, id2 in zip(ids[:-1], ids[1:]):
                x.append(id1)
                y.append(id2)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __len__(self):
        return self.n_data

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 21128
tensor([  12,  644,    1, 2384,   12,  184,    4,    1,   24, 1812,  678,   10,
          51,   36, 1360,   12,    9, 1966,   23, 1035,   20,  655,   12,    2,
         650, 4385,   53,    4, 1599,   41,   14,    4,  135, 1361, 1006,   73,
         847,   59,    8,    6, 1468,    3,    2, 1248, 2330,    8, 2650, 1627,
         577,   19,  529,    3,  407,  997,   48,    3,   62,    2,    7, 2719,
           6,    1, 3935, 4054])
tensor([  53,    4,   39,   10,   16,  406, 1717,  228,  756,   12,    3,  541,
           6,  459,   21,  870,    0,    5, 2628,  271,  222,  179,    1,   60,
         334,    5,  639, 3358,   83,    5, 1161,  405,   26,   10,    5,   10,
        2455,    3,    1,  127,    0,  381,   31,   19,   22,   18,    9,  545,
           0,    2, 1020, 1874,   32,    5, 1926, 1547,  306,    4,  221,  996,
          17, 4136,    0,    6])



----

## モデル構築

マルコフモデルはある状態から次の状態を予測する。言語モデルに当てはめると、ある単語から次の単語を予測する。  
これをNNで実装するので、ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルを出力し、単語の分類問題として扱う。

In [11]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.net = nn.Sequential(
            nn.Linear(vocab_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        x = F.one_hot(x, self.vocab_size).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

通常の分類モデルと同じように学習する。

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_epoch = 0
        for x, t in tqdm(dataloader, desc=f'{epoch}/{n_epochs}', leave=False):
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        loss = loss_epoch/len(dataloader)
        print(f'{epoch}/{n_epochs} loss: {loss}', flush=True)

In [13]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train(model, optimizer, 20)

1/20 loss: 6.614829358737635


2/20 loss: 5.618472875785251


3/20 loss: 5.079186877455236


4/20 loss: 4.561145137804153


5/20 loss: 4.128836105957492


...

16/20 loss: 2.714906456967495


17/20 loss: 2.701112203007378


18/20 loss: 2.6958203949596946


19/20 loss: 2.690627276717356


20/20 loss: 2.6874783939465297
